In [5]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Game window size
WIDTH = 500
HEIGHT = 750
GROUND_LEVEL = 635
GAP_SIZE = 150
PIPE_WIDTH = 70
PIPE_SPEED = -3
BIRD_POS_X = 50

# Create game window
WINDOW = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Flappy Bird")

# Set frame rate
game_clock = pygame.time.Clock()

# Load images
BG_IMAGE = pygame.image.load('background.jpg')
BIRD_SPRITE = pygame.image.load('bird.png')
BIRD_W = BIRD_SPRITE.get_width()
BIRD_H = BIRD_SPRITE.get_height()

# Load fonts
FONT_SCORE = pygame.font.Font('freesansbold.ttf', 32)
FONT_START = pygame.font.Font('freesansbold.ttf', 32)
FONT_OVER_MAIN = pygame.font.Font('freesansbold.ttf', 64)
FONT_OVER_SUB = pygame.font.Font('freesansbold.ttf', 32)

# High score tracking
scores = [0]

# Draw bird at given position
def draw_bird(x, y):
    WINDOW.blit(BIRD_SPRITE, (x, y))

# Draw top and bottom pipes
def draw_pipes(x, top_height):
    bottom_y = top_height + GAP_SIZE
    pygame.draw.rect(WINDOW, (0, 255, 0), (x, 0, PIPE_WIDTH, top_height - 30))         # Top
    pygame.draw.rect(WINDOW, (0, 255, 0), (x - 10, top_height - 30, PIPE_WIDTH + 20, 30))  # Top lip
    pygame.draw.rect(WINDOW, (0, 255, 0), (x, bottom_y, PIPE_WIDTH, GROUND_LEVEL - bottom_y))  # Bottom
    pygame.draw.rect(WINDOW, (0, 255, 0), (x - 10, bottom_y - 30, PIPE_WIDTH + 20, 30))  # Bottom lip

# Check if bird hits pipes
def check_collision(pipe_x, pipe_top, bird_y):
    if BIRD_POS_X + BIRD_W > pipe_x and BIRD_POS_X < pipe_x + PIPE_WIDTH:
        if bird_y <= pipe_top or bird_y + BIRD_H >= pipe_top + GAP_SIZE:
            return True
    return False

# Show current score
def show_score(current_score):
    text = FONT_SCORE.render(f"Score: {current_score}", True, (255, 255, 255))
    WINDOW.blit(text, (10, 10))

# Show start screen
def show_start_screen():
    text = FONT_START.render("TAP SPACE BAR TO START", True, (255, 255, 255))
    WINDOW.blit(text, (50, 200))
    pygame.display.update()

# Show game over screen
def show_game_over(final_score, best_score):
    text1 = FONT_OVER_MAIN.render("GAME OVER", True, (200, 35, 35))
    WINDOW.blit(text1, (50, 300))

    text2 = FONT_OVER_SUB.render(f"SCORE: {final_score}  MAX SCORE: {best_score}", True, (255, 255, 255))
    WINDOW.blit(text2, (50, 400))

    if final_score == best_score:
        text3 = FONT_OVER_SUB.render("NEW HIGH SCORE!!", True, (200, 35, 35))
        WINDOW.blit(text3, (80, 100))

# Game loop flags and state
playing = True
paused = True
hit = False

# Initial positions
bird_y = 300
bird_velocity = 0
pipe_x = WIDTH + 200
pipe_top_height = random.randint(150, 350)
current_score = 0

# Game loop
while playing:
    WINDOW.blit(BG_IMAGE, (0, 0))

    while paused:
        if hit:
            show_game_over(current_score, max(scores))
        show_start_screen()

        for evt in pygame.event.get():
            if evt.type == pygame.QUIT:
                paused = False
                playing = False

            if evt.type == pygame.KEYDOWN and evt.key == pygame.K_SPACE:
                # Reset everything
                bird_y = 300
                bird_velocity = 0
                pipe_x = WIDTH + 200
                pipe_top_height = random.randint(150, 350)
                current_score = 0
                hit = False
                paused = False

    # Event handling
    for evt in pygame.event.get():
        if evt.type == pygame.QUIT:
            playing = False

        if evt.type == pygame.KEYDOWN and evt.key == pygame.K_SPACE:
            bird_velocity = -3
        if evt.type == pygame.KEYUP and evt.key == pygame.K_SPACE:
            bird_velocity = 3

    # Bird movement
    bird_y += bird_velocity

    # Check collision with ground
    if bird_y + BIRD_H >= GROUND_LEVEL:
        hit = True
        scores.append(current_score)
        paused = True
        continue

    # Clamp to top of screen
    bird_y = max(0, bird_y)

    # Move pipes
    pipe_x += PIPE_SPEED

    # Check pipe collision
    if check_collision(pipe_x, pipe_top_height, bird_y):
        hit = True
        scores.append(current_score)
        paused = True
        continue

    # Generate new pipe
    if pipe_x <= -PIPE_WIDTH:
        pipe_x = WIDTH
        pipe_top_height = random.randint(150, 350)
        current_score += 1

    # Drawing
    draw_pipes(pipe_x, pipe_top_height)
    draw_bird(BIRD_POS_X, bird_y)
    show_score(current_score)

    pygame.display.update()
    game_clock.tick(60)

# Exit
pygame.quit()
